In [1]:
import os
import json
import pprint
import csv
import scholarly
import sys
# %matplotlib inline
%load_ext autoreload
%autoreload 2

# test script to pull in metrics from Google Scholar
# does not support author name variations
# issue with enconding in all_sources_meta data leads to exceptions
# example starting point to access scrape info from Google Scolar

# tf.test.gpu_device_name()

ImportError: No module named 'scholarly'

In [2]:
# authors_csv = '../Data/authors.csv'
# authors_last_first = {}
# with open(authors_csv) as fh:
#     for line in fh:
#         splits = line.split(",")
#         fname = splits[3].strip().lower()
#         lname = splits[4].strip().lower()
#         if lname not in authors_last_first:
#             authors_last_first[lname] = set([fname])
#         else:
#             authors_last_first[lname].add(fname)



In [5]:

# class AuthorEntry:
#     def __init__(self, fname, lname, journals=set(), cit=0):
#         self.fname = fname
#         self.lname = lname
#         self.journals = journals
#         self.citiations = cit
#     def __str__(self):
#          return self.fname + ", " + self.lname + "\n" + self.journals + "\n" + self.citiations
#     def __eq__(self, o: object) -> bool:
#         return super().__eq__(o)

#     def __ne__(self, o: object) -> bool:
#         return super().__ne__(o)

#     def __hash__(self) -> int:
#         return super().__hash__()

from bs4 import BeautifulSoup
import urllib.request
author_citations = {};
author_journals = {};
author_articles = {};
article_citations = {};

# sources_csv = '../Data/all_sources_metadata_2020-03-13_clean.csv'
sources_csv = '../Data/all_sources_metadata_2020-03-13_clean_utf8.csv'
row_count = 0
with open(sources_csv) as fh:
    csv_reader = csv.reader(fh, delimiter=',')
#     next(csv_reader)
    # some type of encoding error with the text causes exceptions
    # investigate switch to utf8
    try:
        for row in csv_reader:
            title = row[2].strip()
            abstract = row[7].strip()
            authors = row[9].strip()
            journal = row[10].strip()
    #         print(title, abstract, authors, ' journal: ',  journal)
            author_list = authors.split(";")

            # does not handle variations in author names
            for author in author_list:
                if author in author_citations:
                    pass

                author_query = scholarly.search_author(author)
                try:
                    author_query = next(author_query)
    #             print(search_query)
                except StopIteration:
                    print(f'unable to find information on author {author}')
                    continue

                author_entry ={}

                if author_query.citedby:
                    # not good for same author names
                    author_entry['citedby'] = int(author_query.citedby)

                if author_query.id:
                    url="https://scholar.google.com/citations?user=" + str(author_query.id) + "&hl=en=en"
    #                 print(url)
                    page = urllib.request.urlopen(url)
                    soup = BeautifulSoup(page, 'html.parser')
                    indexes = soup.find_all("td", "gsc_rsb_std")
    #                 print(indexes)
                    h_index = indexes[2].string
                    i10_index = indexes[4].string
                    citations = indexes[0].string

                    h_index_15 = indexes[3].string
                    i10_index_15 = indexes[5].string
                    citations_15 = indexes[1].string

                    author_entry['h_index'] = h_index
                    author_entry['i10_index'] = i10_index
                    author_entry['citations'] = citations
                    author_entry['h_index_15'] = h_index_15
                    author_entry['i10_index_15'] = i10_index_15
                    author_entry['citations_15'] = citations_15

                author_citations[author] = author_entry
    #             break
            if author in author_journals:
                author_journals[author].add(journal)
            else:
                author_journals[author] = set([journal])

            if author in author_articles:
                author_articles[author].add(title)
            else:
                author_articles[author] = set([title])

            pub_query = scholarly.search_pubs_query(title)
            try:
                pub_query = next(pub_query)
    #             print(pub_query)
            except StopIteration:
                print(f'unable to find publication information {pub_query}')
                continue

            if pub_query.citedby:
                    # not good for same author names
                    if author in article_citations:
                        pass
                    article_citations[title] = int(pub_query.citedby)
    #         break;
            row_count += 1
    except:
        e = sys.exc_info()[0]
        print(e)
        print(f'exception caught at row: {row_count}')
        
print('author journals')
pprint.pprint(author_journals)
print('author citations')
pprint.pprint(author_citations)
print('author articles')
pprint.pprint(author_articles)
print('articles citations')
pprint.pprint(article_citations)
print(f'number of rows processed {row_count}')
print(len(author_citations))
      

unable to find publication information <generator object _search_scholar_soup at 0x0000029443F2FA48>
unable to find information on author  Penninger, Josef M.
unable to find information on author  Zhong, Nanshan
unable to find information on author  Slutsky, Arthur S.
unable to find publication information <generator object _search_scholar_soup at 0x00000294441116C8>
unable to find information on author  Feng, Zhimin
unable to find information on author  Wan, Shengqing
unable to find information on author  Huang, Peide
unable to find information on author  Huang, Xuanlin
unable to find publication information <generator object _search_scholar_soup at 0x000002944443E848>
unable to find information on author Linton, M. Natalie
unable to find information on author  Yang, Yichi
unable to find information on author  Hayashi, Katsuma
unable to find publication information <generator object _search_scholar_soup at 0x0000029444948848>
unable to find information on author Deng, Sheng-Qun
unable

In [6]:
# import pickle
# with open('../Data/author_scores.pickle', 'wb') as fh:
#     pickle.dump(author_citations, fh, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
# with open('../Data/author_scores.pickle', 'rb') as fh:
#     author_citations = pickle.load(fh)

In [8]:
with open('../Data/author_scores.csv', 'w', newline='') as csvfile:
    author_writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    author_writer.writerow(['author','h_index',
                           'i10_index',
                           'citations',
                           'h_index_15',
                           'i10_index_15',
                           'citations_15'])

    for author in author_citations:
#         print(author)
        author_entry = author_citations[author]
#         print(author_entry)
        a_splits = author.split(',')
        name = 'UNKNOWN'
        if len(a_splits) > 1:
            lname = a_splits[0]
            fname = a_splits[1]
            name = fname + ' ' + lname
        author_writer.writerow([name, 
                str(author_entry['h_index']),
                author_entry['i10_index'],
                author_entry['citations'],
                author_entry['h_index_15'],
                author_entry['i10_index_15'],
                author_entry['citations_15']])